# Day 12: 使用 Gemini Live API 构建多模态代理

**探索 ADK 双向流：实时多模态 AI 代理开发的可视化指南，基于 WebSockets 和 Gemini Live。**

## 课程来源
- [Advent of Agents 2025 - Day 12](https://adventofagents.com/)
- [ADK Bidi-streaming 文档](https://google.github.io/adk-docs/streaming/)
- [Part 1: ADK Bidi-streaming 介绍](https://google.github.io/adk-docs/streaming/dev-guide/part1/)
- [Part 2: 使用 LiveRequestQueue 发送消息](https://google.github.io/adk-docs/streaming/dev-guide/part2/)
- [Part 3: run_live() 事件处理](https://google.github.io/adk-docs/streaming/dev-guide/part3/)
- [Part 4: 运行配置](https://google.github.io/adk-docs/streaming/dev-guide/part4/)
- [Part 5: 音频、图像和视频](https://google.github.io/adk-docs/streaming/dev-guide/part5/)
- [Gemini Live API 文档](https://ai.google.dev/gemini-api/docs/live)

## 学习目标

1. 理解 Gemini Live API 是什么以及它的重要性
2. 掌握 ADK 的 Bidi-streaming 架构组件
3. 学会使用 `LiveRequestQueue` 进行实时输入
4. 使用 `run_live()` 处理流式事件
5. 配置 `RunConfig` 和 `StreamingMode`
6. 构建处理音频、图像和视频的多模态代理

---
## 1. 什么是 Gemini Live API？

### 从请求-响应到实时流

```
传统请求-响应模式：
┌──────────┐     请求        ┌──────────┐
│  客户端   │ ───────────────▶ │  服务器   │
│          │ ◀─────────────── │          │
└──────────┘     响应        └──────────┘
     │                              │
     │    等待... 等待... 等待...   │
     └──────────────────────────────┘


实时双向流模式：
┌──────────┐ ═══════════════════════ ┌──────────┐
│  客户端   │ ◀────── WebSocket ─────▶ │  服务器   │
│          │       持续双向流         │          │
└──────────┘ ═══════════════════════ └──────────┘
     │                                      │
     │  音频 ──▶  ◀── 音频                 │
     │  视频 ──▶  ◀── 文本                 │
     │  文本 ──▶  ◀── 事件                 │
     └──────────────────────────────────────┘
```

**Gemini Live API** 支持：
- 低延迟双向语音和视频交互
- 自然的、类人的语音对话
- 打断（barge-in）模型响应的能力
- 处理文本、音频和视频输入
- 生成文本和音频输出

### 核心能力

| 功能 | 描述 |
|------|------|
| **语音活动检测 (VAD)** | 自动检测用户何时开始/停止说话 |
| **打断支持** | 用户可以在模型响应中途打断 |
| **原生音频** | 支持 24+ 种语言的自然逼真语音 |
| **工具调用** | 流式对话中的函数调用 |
| **会话管理** | 长时间对话持久化 |
| **多模态输入** | 同时处理音频、视频、图像和文本 |

### 两个 API 平台

| 平台 | 使用场景 | 认证方式 |
|------|----------|----------|
| **Gemini Live API** (Google AI Studio) | 开发、原型设计 | API Key |
| **Vertex AI Live API** (Google Cloud) | 生产环境、企业级 | OAuth/服务账号 |

---
## 2. ADK Bidi-Streaming 架构

### 为什么用 ADK 做流式处理？

虽然 Live API 提供了底层流式技术，但将其集成到生产应用中需要解决复杂问题：

- 管理 WebSocket 连接和重连逻辑
- 编排工具执行和响应处理
- 跨会话持久化对话状态
- 处理音频/视频编解码转换

**ADK 将这些挑战转化为简单的声明式 API。**

### 架构概览

```
┌─────────────────────────────────────────────────────────────────────────┐
│                            客户端应用                                    │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐                     │
│  │    麦克风    │  │    摄像头    │  │   文本输入   │                     │
│  └──────┬──────┘  └──────┬──────┘  └──────┬──────┘                     │
└─────────┼────────────────┼────────────────┼─────────────────────────────┘
          │                │                │
          ▼                ▼                ▼
┌─────────────────────────────────────────────────────────────────────────┐
│                         LiveRequestQueue                                │
│                  持续多模态输入流的缓冲区                                  │
└────────────────────────────────┬────────────────────────────────────────┘
                                 │
                                 ▼
┌─────────────────────────────────────────────────────────────────────────┐
│                              Runner                                      │
│  ┌─────────────────────────────────────────────────────────────────┐   │
│  │                        run_live()                                │   │
│  │  • 从 LiveRequestQueue 消费数据                                  │   │
│  │  • 管理会话状态                                                   │   │
│  │  • 生成 Event 对象                                               │   │
│  └─────────────────────────────────────────────────────────────────┘   │
└────────────────────────────────┬────────────────────────────────────────┘
                                 │
                                 ▼
┌─────────────────────────────────────────────────────────────────────────┐
│                      GeminiLlmConnection                                │
│                  到 Gemini Live API 的 WebSocket 连接                    │
└─────────────────────────────────────────────────────────────────────────┘
```

### 核心组件

| 组件 | 用途 |
|------|------|
| **LiveRequestQueue** | 用于缓冲多模态输入的异步队列 |
| **Runner** | 编排代理会话和工具执行 |
| **run_live()** | 生成流式事件的异步生成器 |
| **RunConfig** | 会话特定配置（模态、VAD 等） |
| **StreamingMode** | BIDI（WebSocket）或 SSE（HTTP） |
| **Event** | 离散更新（响应、工具调用、状态变化） |

---
## 3. 环境设置

In [ ]:
# 安装依赖
# !uv pip install google-adk google-genai google-cloud-aiplatform python-dotenv pyaudio

import sys
import os
from pathlib import Path

# 添加项目根目录到路径
sys.path.insert(0, str(Path.cwd().parent))

# ============================================================
# 选择 API 后端：Google AI Studio 或 Vertex AI
# ============================================================
# 
# 方式 1: Google AI Studio（推荐，你已有付费 API Key）
#   - 使用 GOOGLE_API_KEY 环境变量
#   - 设置 USE_VERTEX_AI = False
#
# 方式 2: Vertex AI（使用 Google Cloud 额度）
#   - 需要先运行: gcloud auth application-default login
#   - 设置 USE_VERTEX_AI = True
#
# ============================================================

USE_VERTEX_AI = False  # 设为 False 使用 Google AI Studio（付费 API Key）

if USE_VERTEX_AI:
    # Vertex AI 配置
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "true"
    os.environ["GOOGLE_CLOUD_PROJECT"] = "gen-lang-client-0262715119"
    os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"
    print("✅ 使用 Vertex AI 后端")
    print(f"   项目 ID: {os.environ['GOOGLE_CLOUD_PROJECT']}")
    print(f"   区域: {os.environ['GOOGLE_CLOUD_LOCATION']}")
else:
    # Google AI Studio 配置（推荐）
    from shared import get_api_key
    os.environ["GOOGLE_API_KEY"] = get_api_key()
    # 确保不使用 Vertex AI
    if "GOOGLE_GENAI_USE_VERTEXAI" in os.environ:
        del os.environ["GOOGLE_GENAI_USE_VERTEXAI"]
    print("✅ 使用 Google AI Studio 后端（付费 API Key）")

---
## 4. LiveRequestQueue：输入通道

### 什么是 LiveRequestQueue？

`LiveRequestQueue` 是一个基于 asyncio 的队列，允许客户端应用无缝地将各种数据类型（文本、音频块、视频帧）按到达顺序入队。

```
┌─────────────────────────────────────────────────────────────────┐
│                      LiveRequestQueue                           │
│                                                                 │
│   ┌─────┐  ┌─────┐  ┌─────┐  ┌─────┐  ┌─────┐                 │
│   │音频 │  │文本 │  │音频 │  │图像 │  │音频 │ ──────▶ Agent   │
│   │ 块  │  │消息 │  │ 块  │  │ 帧  │  │ 块  │                  │
│   └─────┘  └─────┘  └─────┘  └─────┘  └─────┘                 │
│      ▲        ▲        ▲        ▲        ▲                     │
│      │        │        │        │        │                     │
│   麦克风/    用户    麦克风/    摄像头   麦克风/                  │
│   音频流    输入    音频流     帧      音频流                   │
└─────────────────────────────────────────────────────────────────┘
```

**关键点：** 代理的 `run_live()` 从这个队列消费数据，实现近实时处理，无需等待正式的"回合结束"。

In [ ]:
# 理解 LiveRequestQueue 结构

from google.adk.agents import LiveRequestQueue
from google.genai import types

# 创建 LiveRequestQueue
live_request_queue = LiveRequestQueue()

print("✅ LiveRequestQueue 已创建！")
print("\n可用方法：")
print("  - send_content(content): 发送文本或多模态内容")
print("  - send_realtime(blob): 发送音频/视频数据块")
print("  - close(): 发出输入流结束信号")

In [ ]:
# 示例：发送不同类型的输入

import asyncio
from google.genai import types

async def demonstrate_queue_usage():
    """演示向 LiveRequestQueue 发送内容的不同方式。"""
    
    queue = LiveRequestQueue()
    
    # 1. 发送文本内容
    print("1️⃣ 发送文本内容...")
    queue.send_content(
        types.Content(
            role="user",
            parts=[types.Part(text="你好，能帮我吗？")]
        )
    )
    
    # 2. 发送音频数据块（模拟）
    print("2️⃣ 发送音频数据块...")
    # 实际使用中，这将是真实的音频字节
    simulated_audio = b"\x00" * 1024  # 模拟 PCM 音频
    queue.send_realtime(
        types.Blob(
            mime_type="audio/pcm;rate=16000",
            data=simulated_audio
        )
    )
    
    # 3. 发送图像内容
    print("3️⃣ 发送图像内容...")
    # 实际使用中，这将是真实的图像字节
    simulated_image = b"\x89PNG\r\n\x1a\n"  # PNG 头部（模拟）
    queue.send_content(
        types.Content(
            role="user",
            parts=[
                types.Part(
                    inline_data=types.Blob(
                        mime_type="image/png",
                        data=simulated_image
                    )
                ),
                types.Part(text="这张图片里有什么？")
            ]
        )
    )
    
    # 4. 完成后关闭队列
    print("4️⃣ 关闭队列...")
    queue.close()
    
    print("\n✅ 队列演示完成！")

# 运行演示
await demonstrate_queue_usage()

### 音频数据格式

Live API 中的音频始终是 **原始的、小端序、16位 PCM**：

| 参数 | 值 |
|------|----|
| 格式 | PCM（原始） |
| 位深度 | 16 位 |
| 字节序 | 小端序 |
| 声道 | 单声道 (1) |
| 发送采样率 | 16000 Hz |
| 接收采样率 | 24000 Hz |

---
## 5. run_live() 和事件处理

### run_live() 方法

`run_live()` 是一个异步生成器，它：
1. 从 `LiveRequestQueue` 消费数据
2. 维护会话状态
3. 为每次更新生成 `Event` 对象

```python
async for event in runner.run_live(
    session=session,
    live_request_queue=queue,
    run_config=config
):
    # 处理每个事件
    process_event(event)
```

In [ ]:
# 理解事件类型

event_types = """
ADK 流式处理中的事件类型：
═══════════════════════════════════════════════════════════════

1. 文本响应
   ├─ event.content.parts[0].text
   └─ 模型的文本响应（当 response_modalities=["TEXT"] 时）

2. 音频响应
   ├─ event.content.parts[0].inline_data
   │   ├─ .mime_type: "audio/pcm;rate=24000"
   │   └─ .data: 原始 PCM 字节
   └─ 模型的音频响应（当 response_modalities=["AUDIO"] 时）

3. 转录
   ├─ event.content.parts[0].text
   ├─ 输入转录（用户说的话）
   └─ 输出转录（模型音频说的话）

4. 工具调用
   ├─ event.content.parts[0].function_call
   │   ├─ .name: 工具名称
   │   └─ .args: 工具参数
   └─ 模型请求调用工具

5. 回合完成
   ├─ event.turn_complete = True
   └─ 模型完成了响应回合

6. 被打断
   ├─ event.interrupted = True
   └─ 用户在模型响应期间打断（插话）

═══════════════════════════════════════════════════════════════
"""

print(event_types)

In [ ]:
# 事件处理模式（概念示例）

event_handling_code = '''
async def handle_streaming_events(runner, session, queue, config):
    """处理流式事件的模式。"""
    
    async for event in runner.run_live(
        session=session,
        live_request_queue=queue,
        run_config=config
    ):
        # 检查回合完成
        if event.turn_complete:
            print("[回合完成]")
            continue
            
        # 检查被打断
        if event.interrupted:
            print("[被用户打断]")
            continue
        
        # 处理内容部分
        if event.content and event.content.parts:
            for part in event.content.parts:
                
                # 文本响应
                if part.text:
                    print(f"文本: {part.text}")
                
                # 音频响应
                elif part.inline_data:
                    audio_data = part.inline_data.data
                    mime_type = part.inline_data.mime_type
                    print(f"音频: {len(audio_data)} 字节 ({mime_type})")
                    # 播放音频: audio_player.play(audio_data)
                
                # 工具调用
                elif part.function_call:
                    tool_name = part.function_call.name
                    tool_args = part.function_call.args
                    print(f"工具调用: {tool_name}({tool_args})")
'''

print("事件处理模式：")
print(event_handling_code)

---
## 6. RunConfig 和 StreamingMode

### 理解 RunConfig

`RunConfig` 定义每个会话的流式行为：

| 参数 | 描述 | 选项 |
|------|------|------|
| `streaming_mode` | 连接类型 | `BIDI`, `SSE`, `NONE` |
| `response_modalities` | 输出格式 | `["TEXT"]` 或 `["AUDIO"]` |
| `input_audio_transcription` | 转录用户语音 | 启用/禁用 |
| `output_audio_transcription` | 转录模型语音 | 启用/禁用 |
| `session_resumption` | 处理重连 | 启用/禁用 |

In [ ]:
# RunConfig 示例

from google.adk.agents.run_config import RunConfig, StreamingMode
from google.genai import types

# 配置 1: 纯文本流
text_config = RunConfig(
    streaming_mode=StreamingMode.BIDI,
    response_modalities=["TEXT"],  # 仅文本输出
)
print("1️⃣ 纯文本配置：")
print(f"   模式: {text_config.streaming_mode}")
print(f"   模态: {text_config.response_modalities}")

# 配置 2: 带转录的音频流
audio_config = RunConfig(
    streaming_mode=StreamingMode.BIDI,
    response_modalities=["AUDIO"],  # 音频输出
    input_audio_transcription=types.AudioTranscriptionConfig(),  # 启用输入转录
    output_audio_transcription=types.AudioTranscriptionConfig(),  # 启用输出转录
)
print("\n2️⃣ 带转录的音频配置：")
print(f"   模式: {audio_config.streaming_mode}")
print(f"   模态: {audio_config.response_modalities}")
print(f"   输入转录: 已启用")
print(f"   输出转录: 已启用")

# 配置 3: 带会话恢复（用于长对话）
resumable_config = RunConfig(
    streaming_mode=StreamingMode.BIDI,
    response_modalities=["AUDIO"],
    session_resumption=types.SessionResumptionConfig(),  # 自动重连
)
print("\n3️⃣ 可恢复会话配置：")
print(f"   模式: {resumable_config.streaming_mode}")
print(f"   会话恢复: 已启用（处理 10 分钟限制）")

### StreamingMode: BIDI vs SSE

| 特性 | BIDI (WebSocket) | SSE (HTTP) |
|------|------------------|------------|
| **连接** | 持久 WebSocket | HTTP 流 |
| **方向** | 双向 | 仅服务器 → 客户端 |
| **实时音频** | 完全支持 | 有限 |
| **打断** | 支持 | 不支持 |
| **使用场景** | 语音对话 | 文本流 |
| **使用的 API** | Live API | 标准 generate_content_async |

```
BIDI 模式 (WebSocket):
┌────────┐  音频/视频/文本   ┌────────┐
│ 客户端  │ ◀═══════════════▶ │ Gemini │
└────────┘    持续双向流     └────────┘

SSE 模式 (HTTP):
┌────────┐      请求         ┌────────┐
│ 客户端  │ ─────────────────▶ │ Gemini │
│        │ ◀═══════════════   │        │
└────────┘    流式响应       └────────┘
```

### 响应模态限制

**重要：** Gemini Live API 和 Vertex AI Live API 都有一个限制：

> **每个会话只能有一种响应模态**

- 纯文本应用使用 `["TEXT"]`
- 语音应用使用 `["AUDIO"]`
- 不能在同一会话中混合使用

---
## 7. 构建完整的流式代理

### 最小文本流示例

In [ ]:
# 完整的流式代理示例

import asyncio
from google.adk.agents import Agent, LiveRequestQueue
from google.adk.runners import InMemoryRunner
from google.adk.agents.run_config import RunConfig, StreamingMode
from google.genai import types

# ============================================================
# Live 模型选择（重要！）
# ============================================================
# 
# Google AI Studio 支持的 Live 模型：
#   - gemini-2.5-flash-native-audio-latest (仅音频)
#   - gemini-2.5-flash-native-audio-preview-12-2025 (仅音频)
#
# 📌 重要限制：
#   - Native Audio 模型只支持 response_modalities=["AUDIO"]
#   - 不能使用 TEXT 模态！
#   - Live API 需要实时 WebSocket 连接
#
# ============================================================

if USE_VERTEX_AI:
    LIVE_MODEL = "gemini-2.0-flash-live-001"
    SUPPORTS_TEXT = True  # Vertex AI 可能支持文本
else:
    # Google AI Studio - Native Audio 模型（推荐）
    LIVE_MODEL = "gemini-2.5-flash-native-audio-preview-12-2025"
    SUPPORTS_TEXT = False  # Native Audio 只支持音频输出

streaming_agent = Agent(
    model=LIVE_MODEL,
    name="streaming_assistant",
    description="实时流式助手",
    instruction="""
    你是一个有帮助的流式助手。
    回复要简洁自然。
    当用户提问时，提供清晰的答案。
    请用中文回复。
    """,
)

print("✅ 代理已创建: streaming_assistant")
print(f"   模型: {streaming_agent.model}")
print(f"   支持文本模式: {SUPPORTS_TEXT}")
if not SUPPORTS_TEXT:
    print("\n📌 注意: Native Audio 模型只能输出音频，将使用 output_audio_transcription 获取文本转录")

In [ ]:
# 运行流式代理

async def run_streaming_demo():
    """演示 ADK 流式处理。"""
    
    # 创建 runner 和 session
    runner = InMemoryRunner(agent=streaming_agent, app_name="streaming_demo")
    session = await runner.session_service.create_session(
        app_name="streaming_demo",
        user_id="demo_user"
    )
    
    # 创建请求队列
    queue = LiveRequestQueue()
    
    # 根据模型能力选择配置
    if SUPPORTS_TEXT:
        # Vertex AI: 支持文本模式
        config = RunConfig(
            streaming_mode=StreamingMode.BIDI,
            response_modalities=["TEXT"],
        )
        print("📝 使用文本模式")
    else:
        # Google AI Studio Native Audio: 只支持音频
        config = RunConfig(
            streaming_mode=StreamingMode.BIDI,
            response_modalities=["AUDIO"],
            output_audio_transcription=types.AudioTranscriptionConfig(),
        )
        print("🎤 使用音频模式（带转录）")
    
    # 发送消息
    print("📤 发送消息: '法国的首都是哪里？'")
    queue.send_content(
        types.Content(
            role="user",
            parts=[types.Part(text="法国的首都是哪里？")]
        )
    )
    
    # 关闭队列表示发送完成
    queue.close()
    
    # 处理流式事件
    print("\n📥 流式响应：")
    print("-" * 40)
    
    full_response = ""
    try:
        async for event in runner.run_live(
            session=session,
            live_request_queue=queue,
            run_config=config
        ):
            if event.turn_complete:
                print("\n[回合完成]")
                break
                
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        print(part.text, end="", flush=True)
                        full_response += part.text
                    if part.inline_data:
                        print(f"[音频 {len(part.inline_data.data)} bytes]", end="", flush=True)
    except Exception as e:
        error_msg = str(e)
        print(f"\n❌ 错误: {type(e).__name__}")
        
        # 提供有用的错误诊断
        if "1008" in error_msg or "policy violation" in error_msg:
            if "not found" in error_msg.lower() or "Publisher Model" in error_msg:
                print("\n📌 诊断: 模型不可用或未启用")
                print("   可能的原因:")
                print("   1. Live API 模型需要在项目中启用")
                print("   2. 模型名称已更改")
                print("   3. 区域不支持此模型")
                print("\n   建议解决方案:")
                print("   1. 访问 Google Cloud Console -> Vertex AI")
                print("   2. 检查 Model Garden 中的可用模型")
                print("   3. 尝试使用 Google AI Studio 的 Live 体验")
            elif "Permission denied" in error_msg:
                print("\n📌 诊断: 权限不足")
                print("   建议: 确保账号有 Vertex AI User 角色")
        elif "429" in error_msg or "RESOURCE_EXHAUSTED" in error_msg:
            print("\n📌 诊断: API 配额已用完")
            print("   建议: 等待配额恢复或升级计划")
        elif "Cannot extract voices" in error_msg:
            print("\n📌 诊断: Native Audio 模型不支持 TEXT 模态")
            print("   建议: 使用 response_modalities=['AUDIO']")
        else:
            print(f"\n   详情: {error_msg[:200]}...")
    
    print("\n" + "-" * 40)
    if full_response:
        print(f"📊 响应总长度: {len(full_response)} 字符")

# 运行演示
print("🚀 开始流式演示...")
print("=" * 50)
await run_streaming_demo()

In [ ]:
# ============================================================
# 备用方案：使用标准 run_async() 测试代理
# ============================================================
# 如果 Live API 不可用，可以使用这个简单测试来验证代理配置

async def test_agent_basic():
    """使用标准 API 测试代理（不使用 Live API）。"""
    from google.adk.runners import InMemoryRunner
    from google.genai.types import Content, Part
    
    # 使用普通模型（非 Live）
    basic_agent = Agent(
        model="gemini-2.0-flash",  # 标准模型
        name="basic_assistant",
        instruction="你是一个有帮助的助手。请用中文简洁回复。",
    )
    
    runner = InMemoryRunner(agent=basic_agent, app_name="basic_test")
    session = await runner.session_service.create_session(
        app_name="basic_test",
        user_id="test_user"
    )
    
    print("📤 测试基本 Agent（非 Live API）...")
    print("-" * 40)
    
    # 使用标准 run_async 方法 - 这是一个 async generator
    async for event in runner.run_async(
        user_id="test_user",
        session_id=session.id,
        new_message=Content(
            role="user",
            parts=[Part(text="法国的首都是哪里？用一句话回答。")]
        )
    ):
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"✅ 响应: {part.text}")
    
    print("-" * 40)
    print("🎉 基本代理测试成功！")

# 运行基本测试
print("🧪 测试基本 Agent 功能...")
await test_agent_basic()

---
## 8. 多模态输入：图像和视频

### 向流式代理发送图像

In [ ]:
# 带图像的多模态流式示例

import base64
from pathlib import Path

async def run_image_streaming_demo(image_path: str = None):
    """演示图像 + 文本流。"""
    
    # 创建 runner 和 session
    runner = InMemoryRunner(agent=streaming_agent, app_name="multimodal_demo")
    session = await runner.session_service.create_session(
        app_name="multimodal_demo",
        user_id="demo_user"
    )
    
    queue = LiveRequestQueue()
    config = RunConfig(
        streaming_mode=StreamingMode.BIDI,
        response_modalities=["TEXT"],
    )
    
    # 准备内容部分
    parts = []
    
    if image_path and Path(image_path).exists():
        # 加载真实图像
        with open(image_path, "rb") as f:
            image_data = f.read()
        mime_type = "image/png" if image_path.endswith(".png") else "image/jpeg"
        parts.append(
            types.Part(
                inline_data=types.Blob(
                    mime_type=mime_type,
                    data=image_data
                )
            )
        )
        print(f"📷 已加载图像: {image_path}")
    else:
        print("ℹ️ 未提供图像，使用纯文本查询")
    
    parts.append(types.Part(text="描述你在这张图片中看到的内容。"))
    
    # 发送多模态内容
    queue.send_content(
        types.Content(role="user", parts=parts)
    )
    queue.close()
    
    # 处理响应
    print("\n📥 流式响应：")
    print("-" * 40)
    
    async for event in runner.run_live(
        session=session,
        live_request_queue=queue,
        run_config=config
    ):
        if event.turn_complete:
            print("\n[回合完成]")
            break
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(part.text, end="", flush=True)

# 不带图像运行（演示）
# 要用图像测试，请传入: await run_image_streaming_demo("path/to/image.png")
print("📷 图像流示例已准备就绪。")
print("用法: await run_image_streaming_demo('path/to/image.png')")

### 使用流式工具进行视频流处理

ADK 支持专门用于视频处理的 **流式工具 (Streaming Tools)**：

```python
def analyze_video_frames(input_stream: LiveRequestQueue) -> str:
    """处理视频帧的流式工具。
    
    `input_stream: LiveRequestQueue` 参数是特殊的：
    - ADK 自动注入实时请求队列
    - 工具循环，通过 input_stream.get() 消费帧
    - 当满足条件时生成结果
    """
    # 实时处理视频帧
    while True:
        frame = input_stream.get()
        if frame is None:
            break
        # 分析帧...
        if condition_met:
            return "分析结果"
```

> **关键点：** 参数名 `input_stream: LiveRequestQueue` 是保留的。当存在时，ADK 会自动注入队列。

---
## 9. 音频流：语音对话

### 音频流架构

```
┌─────────────────────────────────────────────────────────────────┐
│                       音频流管道                                 │
│                                                                  │
│   ┌─────────────┐     ┌──────────────┐     ┌─────────────┐     │
│   │    麦克风    │────▶│  音频编码器   │────▶│ LiveRequest │     │
│   │  (PyAudio)  │     │ (PCM 16kHz)  │     │    Queue    │     │
│   └─────────────┘     └──────────────┘     └──────┬──────┘     │
│                                                    │            │
│                                                    ▼            │
│                                             ┌──────────────┐   │
│                                             │  run_live()  │   │
│                                             │   (Agent)    │   │
│                                             └──────┬───────┘   │
│                                                    │            │
│   ┌─────────────┐     ┌──────────────┐     ┌──────▼───────┐   │
│   │    扬声器    │◀────│  音频解码器   │◀────│    Events    │   │
│   │  (PyAudio)  │     │ (PCM 24kHz)  │     │  (音频数据)   │   │
│   └─────────────┘     └──────────────┘     └──────────────┘   │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# 音频流代理示例（概念性 - 需要 PyAudio）

audio_agent_code = '''
import asyncio
import pyaudio
from google.adk.agents import Agent, LiveRequestQueue
from google.adk.runners import InMemoryRunner
from google.adk.agents.run_config import RunConfig, StreamingMode
from google.genai import types

# 音频配置
SEND_SAMPLE_RATE = 16000  # 发送到 Gemini
RECEIVE_SAMPLE_RATE = 24000  # 从 Gemini 接收
CHANNELS = 1
FORMAT = pyaudio.paInt16
CHUNK_SIZE = 1024

# 代理定义
voice_agent = Agent(
    model="gemini-2.5-flash-native-audio-preview-12-2025",  # Live 模型
    name="voice_assistant",
    instruction="你是一个友好的语音助手。请自然地、对话式地回复。",
)

async def audio_streaming_conversation():
    """与代理进行语音对话。"""
    
    # 设置 PyAudio
    p = pyaudio.PyAudio()
    
    # 输入流（麦克风）
    mic_stream = p.open(
        format=FORMAT,
        channels=CHANNELS,
        rate=SEND_SAMPLE_RATE,
        input=True,
        frames_per_buffer=CHUNK_SIZE
    )
    
    # 输出流（扬声器）
    speaker_stream = p.open(
        format=FORMAT,
        channels=CHANNELS,
        rate=RECEIVE_SAMPLE_RATE,
        output=True
    )
    
    # ADK 设置
    runner = InMemoryRunner(agent=voice_agent, app_name="voice_app")
    session = await runner.session_service.create_session(
        app_name="voice_app",
        user_id="user"
    )
    
    queue = LiveRequestQueue()
    config = RunConfig(
        streaming_mode=StreamingMode.BIDI,
        response_modalities=["AUDIO"],
        input_audio_transcription=types.AudioTranscriptionConfig(),
        output_audio_transcription=types.AudioTranscriptionConfig(),
    )
    
    # 后台任务：从麦克风发送音频到队列
    async def send_audio():
        while True:
            audio_data = mic_stream.read(CHUNK_SIZE, exception_on_overflow=False)
            queue.send_realtime(
                types.Blob(mime_type="audio/pcm;rate=16000", data=audio_data)
            )
            await asyncio.sleep(0.01)
    
    # 开始发送音频
    send_task = asyncio.create_task(send_audio())
    
    # 处理事件
    async for event in runner.run_live(
        session=session,
        live_request_queue=queue,
        run_config=config
    ):
        if event.content and event.content.parts:
            for part in event.content.parts:
                # 播放音频输出
                if part.inline_data and "audio" in part.inline_data.mime_type:
                    speaker_stream.write(part.inline_data.data)
                
                # 打印转录
                if part.text:
                    print(f"[转录] {part.text}")
        
        if event.interrupted:
            print("[用户打断]")
    
    # 清理
    send_task.cancel()
    mic_stream.stop_stream()
    mic_stream.close()
    speaker_stream.stop_stream()
    speaker_stream.close()
    p.terminate()
'''

print("🎤 音频流代理示例：")
print(audio_agent_code)

---
## 10. 会话管理和重连

### 连接 vs 会话持续时间

| 概念 | 持续时间 | 说明 |
|------|----------|------|
| **连接** | ~10 分钟 | WebSocket 自动关闭 |
| **会话** | 无限制 | 逻辑对话持久化 |

### 会话恢复

当你启用 `session_resumption` 时，ADK 会自动处理重连：

```python
config = RunConfig(
    streaming_mode=StreamingMode.BIDI,
    response_modalities=["AUDIO"],
    session_resumption=types.SessionResumptionConfig(),  # 启用自动重连
)
```

**ADK 自动处理的内容：**
- 检测连接关闭
- 缓存恢复句柄
- 后台无缝重连
- 突破 10 分钟限制继续会话

---
## 11. 流式模式中的工具

### 流式处理期间的工具调用

流式代理可以像非流式代理一样使用函数调用：

In [ ]:
# 带工具的流式代理

def get_weather(city: str) -> dict:
    """获取城市的当前天气。"""
    # 模拟天气数据
    weather_data = {
        "东京": {"temp": 22, "condition": "晴朗"},
        "伦敦": {"temp": 15, "condition": "多云"},
        "纽约": {"temp": 18, "condition": "局部多云"},
        "北京": {"temp": 20, "condition": "晴"},
        "上海": {"temp": 24, "condition": "多云"},
    }
    if city in weather_data:
        return {"city": city, **weather_data[city]}
    return {"city": city, "temp": 20, "condition": "未知"}

def search_restaurants(cuisine: str, location: str) -> list:
    """按菜系和位置搜索餐厅。"""
    # 模拟餐厅数据
    return [
        {"name": f"最佳{cuisine}餐厅", "rating": 4.5, "location": location},
        {"name": f"{cuisine}之家", "rating": 4.2, "location": location},
    ]

# 带工具的代理
tool_streaming_agent = Agent(
    model=LIVE_MODEL,  # 使用上面定义的 Live 模型
    name="assistant_with_tools",
    instruction="""
    你是一个有帮助的助手，可以访问天气和餐厅信息。
    当用户询问天气或餐厅时，使用可用的工具。
    请用中文回复。
    """,
    tools=[get_weather, search_restaurants],
)

print("✅ 带工具的流式代理已创建！")
print(f"   工具: {[t.__name__ for t in tool_streaming_agent.tools]}")

In [ ]:
# 运行带工具的流式代理

async def run_tool_streaming_demo():
    """演示流式模式中的工具调用。"""
    
    runner = InMemoryRunner(agent=tool_streaming_agent, app_name="tool_demo")
    session = await runner.session_service.create_session(
        app_name="tool_demo",
        user_id="demo_user"
    )
    
    queue = LiveRequestQueue()
    config = RunConfig(
        streaming_mode=StreamingMode.BIDI,
        response_modalities=["TEXT"],
    )
    
    # 应该触发工具使用的查询
    print("📤 发送: '东京的天气怎么样？推荐一些那里的寿司餐厅'")
    queue.send_content(
        types.Content(
            role="user",
            parts=[types.Part(text="东京的天气怎么样？推荐一些那里的寿司餐厅")]
        )
    )
    queue.close()
    
    print("\n📥 流式响应（带工具调用）：")
    print("-" * 50)
    
    async for event in runner.run_live(
        session=session,
        live_request_queue=queue,
        run_config=config
    ):
        if event.turn_complete:
            print("\n[回合完成]")
            break
            
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(part.text, end="", flush=True)
                elif part.function_call:
                    print(f"\n[工具调用: {part.function_call.name}({part.function_call.args})]")

# 运行演示
await run_tool_streaming_demo()

---
## 12. 生产环境架构

### 生产环境推荐架构

```
┌─────────────────────────────────────────────────────────────────────────┐
│                           生产环境架构                                   │
│                                                                          │
│   ┌─────────────┐        ┌─────────────────────────────────────────┐   │
│   │   客户端    │        │              后端服务器                   │   │
│   │  (浏览器/   │        │                                          │   │
│   │   移动端)   │        │  ┌─────────────────────────────────┐    │   │
│   │             │        │  │        WebSocket 处理器          │    │   │
│   │ ┌─────────┐ │  WS    │  │  • 接收客户端音频/视频          │    │   │
│   │ │  媒体   │ │◀═════▶│  │  • 发送代理响应                  │    │   │
│   │ │  捕获   │ │        │  └──────────────┬──────────────────┘    │   │
│   │ └─────────┘ │        │                 │                        │   │
│   │             │        │                 ▼                        │   │
│   │ ┌─────────┐ │        │  ┌─────────────────────────────────┐    │   │
│   │ │  音频   │ │        │  │      ADK Runner + Agent         │    │   │
│   │ │  播放器 │ │        │  │  • LiveRequestQueue             │    │   │
│   │ └─────────┘ │        │  │  • run_live() 事件循环          │    │   │
│   └─────────────┘        │  │  • 会话管理                      │    │   │
│                          │  └──────────────┬──────────────────┘    │   │
│                          │                 │                        │   │
│                          │                 ▼                        │   │
│                          │  ┌─────────────────────────────────┐    │   │
│                          │  │     Gemini Live API (WS)        │    │   │
│                          │  └─────────────────────────────────┘    │   │
│                          └─────────────────────────────────────────┘   │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### 关键考虑因素

| 方面 | 建议 |
|------|------|
| **认证** | 使用后端代理，永远不要向客户端暴露 API 密钥 |
| **连接管理** | 启用 session_resumption 处理长对话 |
| **错误处理** | 实现指数退避的重连逻辑 |
| **音频编解码** | 客户端传输使用 Opus，转换为 PCM 发送到 API |
| **扩展** | 后端服务器使用水平扩展 |

---
## 13. 核心要点

### Gemini Live API

| 特性 | 描述 |
|------|------|
| **协议** | WebSocket 双向流 |
| **能力** | 音频/视频输入，音频/文本输出 |
| **关键功能** | 打断（barge-in）支持 |
| **连接限制** | ~10 分钟（使用会话恢复） |

### 可用的 Live 模型（2025年12月）

| 模型 | 说明 |
|------|------|
| `gemini-2.5-flash-native-audio-latest` | 最新版本 |
| `gemini-2.5-flash-native-audio-preview-12-2025` | 2025年12月预览版 |

> **重要：** Native Audio 模型只支持 `response_modalities=["AUDIO"]`，不支持纯文本输出！

### ADK 流式组件

```python
# 核心导入
from google.adk.agents import Agent, LiveRequestQueue
from google.adk.runners import InMemoryRunner
from google.adk.agents.run_config import RunConfig, StreamingMode

# 工作流程
1. 创建支持 live 的 Agent
2. 创建 Runner 和 Session
3. 创建 LiveRequestQueue 用于输入
4. 配置 RunConfig（模态必须是 AUDIO）
5. 使用 run_live() 处理事件
6. 处理音频/转录/工具事件
```

### 常见错误及解决方案

| 错误 | 原因 | 解决方案 |
|------|------|----------|
| `Cannot extract voices from a non-audio request` | 使用了 TEXT 模态 | 改为 `["AUDIO"]` |
| `429 RESOURCE_EXHAUSTED` | API 配额用完 | 等待配额恢复或升级计划 |
| `1008 policy violation` | 模型名称错误 | 使用正确的 Live 模型名 |
| 连接超时 | 网络问题或服务器负载 | 重试或稍后再试 |

### 最佳实践

| 实践 | 原因 |
|------|------|
| Native Audio 模型使用 `["AUDIO"]` | 唯一支持的输出模态 |
| 启用 `output_audio_transcription` | 获取音频的文本转录 |
| 启用 `session_resumption` | 处理 10 分钟连接限制 |
| 异步处理事件 | 保持响应性 |
| 使用后端代理 | 安全的 API 密钥管理 |

### 快速体验

最简单的方式是在 Google AI Studio 直接体验：

1. 访问 https://aistudio.google.com
2. 点击顶部 **"Live"** 标签
3. 点击 **"Talk"** 按钮
4. 开始语音对话！

---
## 14. 资源链接

### 官方文档
- [ADK Bidi-streaming 文档](https://google.github.io/adk-docs/streaming/)
- [Part 1: ADK Bidi-streaming 介绍](https://google.github.io/adk-docs/streaming/dev-guide/part1/)
- [Part 2: 使用 LiveRequestQueue 发送消息](https://google.github.io/adk-docs/streaming/dev-guide/part2/)
- [Part 3: run_live() 事件处理](https://google.github.io/adk-docs/streaming/dev-guide/part3/)
- [Part 4: 运行配置](https://google.github.io/adk-docs/streaming/dev-guide/part4/)
- [Part 5: 音频、图像和视频](https://google.github.io/adk-docs/streaming/dev-guide/part5/)
- [流式工具](https://google.github.io/adk-docs/streaming/streaming-tools/)

### Gemini Live API
- [Live API 入门](https://ai.google.dev/gemini-api/docs/live)
- [Live API 能力指南](https://ai.google.dev/gemini-api/docs/live-guide)
- [Vertex AI Live API](https://cloud.google.com/vertex-ai/generative-ai/docs/live-api)

### 教程和示例
- [构建流式代理（快速入门）](https://google.github.io/adk-docs/get-started/streaming/)
- [自定义音频流应用（SSE）](https://google.github.io/adk-docs/streaming/custom-streaming/)
- [Google ADK + Vertex AI Live API](https://medium.com/google-cloud/google-adk-vertex-ai-live-api-125238982d5e)
- [使用 ADK 构建语音流式 AI 代理](https://medium.com/google-cloud/build-your-voice-streaming-ai-agents-with-adk-and-google-search-grounding-0cfcdec63d1e)
- [Multimodal Live API 快速入门 Colab](https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Get_started_LiveAPI.ipynb)

### 深度文章
- [超越请求-响应：构建实时双向流架构](https://developers.googleblog.com/en/beyond-request-response-architecting-real-time-bidirectional-streaming-multi-agent-system/)
- [Vertex AI 中的 Gemini Live API 开发者指南](https://cloud.google.com/blog/topics/developers-practitioners/how-to-use-gemini-live-api-native-audio-in-vertex-ai)
- [ADK Bidi-streaming：可视化指南 (Medium)](https://medium.com/google-cloud/adk-bidi-streaming-a-visual-guide-to-real-time-multimodal-ai-agent-development-62dd08c81399)